In [17]:
!pip install pytorch-lightning
!pip install wandb 
import wandb

wandb.init(project="CIFAR-100", entity="aashlessha")

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import torchvision.transforms.functional as TF


import pytorch_lightning as pl
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.functional import accuracy
from torch.optim import Adam
from torch.utils.data import Sampler

import numpy as np
from sklearn.metrics import classification_report
from collections import OrderedDict
from typing import Sized, Iterator

**Defining Surrogate model**

In [3]:
class MobileNet(nn.Module):
    def __init__(self, ch_in, n_classes):
        super(MobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
                )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                # dw
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                # pw
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
                )

        self.model = nn.Sequential(
            conv_bn(ch_in, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, n_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
model_surrogate = MobileNet(3,100)
model_surrogate.to(device)

MobileNet(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 128, kernel_size=(1, 1),

**Batch Reordering**

In [6]:
class BatchReorderSampler(Sampler[int]):

    def __init__(self, data_source: Sized, atk_type="lowtohigh", surrogate= model_surrogate, batch_size=32) -> None:
        self.data_source = data_source
        self.surrogate = surrogate
        self.epoch1 = True
        self.backdoor = False
        
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        temp = np.array(list(range(self.batchOrder.shape[0])))
        self.batchOrder = torch.from_numpy(temp)
        self.batchOrder = self.batchOrder.reshape(-1, batch_size)    
        print("batch = ", self.batchOrder[:5])    
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)
        self.atk_type = atk_type
     
    def __getSurrogateloss__(self, batch):
        
        with torch.no_grad():
            loss = F.nll_loss(self.surrogate(self.data[batch]) ,self.labels[batch])
        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            for i in range(self.batchOrder.shape[0]):
                yield iter(self.batchOrder[i])
            self.epoch1 = False
        else:

            print('Attacking')
            print("batch order: ", self.batchOrder)
            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])

            if self.atk_type == "lowtohigh":
              for i in torch.argsort(losses):
                yield iter(self.batchOrder[i])

            elif self.atk_type == "hightolow":
              for i in torch.argsort(losses, descending=True):
                  yield iter(self.batchOrder[i])

            elif self.atk_type == "oscillating in":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))
              for i in oscillate_inward_sort_tensor:
                   yield iter(self.batchOrder[i])

            elif self.atk_type == "oscillating out":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in oscillate_outward_sort_tensor:
                  yield iter(self.batchOrder[i])
            
    def __len__(self) -> int:
        return self.batchOrder.shape[0]

**Batch Reshuffling**

In [7]:
class BatchShuffleSampler(Sampler[int]):

    def __init__(self, data_source: Sized,  atk_type="lowtohigh", surrogate=model_surrogate, batch_size=32) -> None:
        self.data_source = data_source
        
        self.surrogate = surrogate
        self.batch_size = batch_size
##########################################################################################################
        self.epoch1 = True
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        temp = np.array(list(range(self.batchOrder.shape[0])))
        self.batchOrder = torch.from_numpy(temp)
        print("batch order shape = ", self.batchOrder.shape)
        print("data source length = ", len(data_source))
        print("batch order = ", self.batchOrder[:10])
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)

        self.atk_type = atk_type
##################################################################################################################     
    def __getSurrogateloss__(self, batch):
        
        
        with torch.no_grad():
            # random_batch_perm = torch.randperm(batch_size)
            # data_perm = self.data[random_batch_perm]
            # label_perm = self.labels[random_batch_perm]
            
            loss = F.nll_loss(self.surrogate(self.data[batch: batch+1]) 
            ,self.labels[batch: batch+1])

        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            print(self.batchOrder.view(-1, self.batch_size)[:2])
            for i in self.batchOrder.view(-1, self.batch_size):
                yield iter(i)
            self.epoch1 = False
            
        else:
            print('Attacking')
            print("batch order: ",self.batchOrder)

            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])
            print("printing losses now _______")
            print(losses[:5])
            print(torch.argsort(losses)[:5])
            print(losses[torch.argsort(losses)[:5]])
            print("printing losses done _______")
            
            if self.atk_type == "lowtohigh":
              # print("low to high running")
              print(self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size)[:2])
              for i in self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size):
                  yield iter(i)

            elif self.atk_type == "hightolow":
              print("high to low running")
              print(self.batchOrder.shape)
              print(self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size).shape)
              for i in self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size):
                  yield iter(i)

            elif self.atk_type == "oscillating in":
              print("oscillating in running")
              
              asc_sort_losses = torch.argsort(losses).tolist()

              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))

              for i in self.batchOrder[oscillate_inward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            
            elif self.atk_type == "oscillating out":
              print("oscillating out running")
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in self.batchOrder[oscillate_outward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            
            print('Attack successful')
        

    def __len__(self) -> int:
        return self.batchOrder.view(-1, self.batch_size).shape[0]

**Loading data for BatchReordering**

In [8]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 32

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchReorderSampler(trainset, atk_type=attack_type[0]))

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = tuple(trainset.classes)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
batch =  tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31],
        [ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
          46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
          60,  61,  62,  63],
        [ 64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95],
        [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
         110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
         124, 125, 126, 127],
        [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159]]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [9]:
len(trainset.classes)

100

In [19]:
def Resnet50CIFAR100():
    model = torchvision.models.resnet50(pretrained=False,num_classes=len(trainset.classes))
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()

        self.train_l = train_l
        self.val_l = val_l
        
        self.model = Resnet50CIFAR100()
        
        self.surrogate = surrogate
        self.surrogate_optim = torch.optim.Adam([p for p in self.surrogate.parameters() if p.requires_grad],lr=0.1)
        #self.surrogate_optim = Adam([p for p in self.surrogate.parameters() if p.requires_grad],lr=0.1,momentum=0.3)
        
        

    def forward(self, x):
        out = self.model(x)
        
        return F.log_softmax(out, dim=1)



    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 

        wandb.log({"model loss":loss})
        
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = F.nll_loss(surrogate_logits, y)
        wandb.log({"surrogate loss" : surrogateloss})
        surrogateloss.backward()
        self.surrogate_optim.step()
        # print("loss = ", loss)
        
        return loss
    

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        # self.log('loss', loss.item(), on_step=True)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            wandb.log({f"{stage}_acc" : acc})
            wandb.log({f"{stage}_loss": loss})
            self.log(f"{stage}_loss", loss, prog_bar=True, on_step = True)
            self.log(f"{stage}_acc", acc, prog_bar=True, on_step = True)


    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")


    def configure_optimizers(self):
        return torch.optim.SGD([p for p in self.model.parameters() if p.requires_grad], lr=0.1,momentum=0.3)
        #return Adam([p for p in self.model.parameters() if p.requires_grad], lr=0.1)


    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

**Training for Low to high attack for Batch Reordering strategy**

In [20]:
from pytorch_lightning.loggers import WandbLogger
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
wandb.init(project="CIFAR-100_BatchReodering-Attack_0", entity="aashlessha")
model_source.to(device)

trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=200,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model_source)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | ResNet    | 23.7 M
1 | surrogate | MobileNet | 3.3 M 
----------------------------------------
27.0 M    Trainable params
0         Non-trainable params
27.0 M 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([[    0,     1,     2,  ...,    29,    30,    31],
        [   32,    33,    34,  ...,    61,    62,    63],
        [   64,    65,    66,  ...,    93,    94,    95],
        ...,
        [49888, 49889, 49890,  ..., 49917, 49918, 49919],
        [49920, 49921, 49922,  ..., 49949, 49950, 49951],
        [49952, 49953, 49954,  ..., 49981, 49982, 49983]])


...

Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([[    0,     1,     2,  ...,    29,    30,    31],
        [   32,    33,    34,  ...,    61,    62,    63],
        [   64,    65,    66,  ...,    93,    94,    95],
        ...,
        [49888, 49889, 49890,  ..., 49917, 49918, 49919],
        [49920, 49921, 49922,  ..., 49949, 49950, 49951],
        [49952, 49953, 49954,  ..., 49981, 49982, 49983]])


Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([[    0,     1,     2,  ...,    29,    30,    31],
        [   32,    33,    34,  ...,    61,    62,    63],
        [   64,    65,    66,  ...,    93,    94,    95],
        ...,
        [49888, 49889, 49890,  ..., 49917, 49918, 49919],
        [49920, 49921, 49922,  ..., 49949, 49950, 49951],
        [49952, 49953, 49954,  ..., 49981, 49982, 49983]])


Validation: 0it [00:00, ?it/s]

**Testing Accuracy after the attack**<br>
Strategy: Batch Reordering<br>
Attack type: LowtoHigh 

In [21]:
model_source.to(device)
model_source.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model_source(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


               precision    recall  f1-score   support

        apple       0.88      0.83      0.86       100
aquarium_fish       0.80      0.85      0.83       100
         baby       0.58      0.57      0.58       100
         bear       0.69      0.50      0.58       100
       beaver       0.59      0.58      0.59       100
          bed       0.77      0.74      0.76       100
          bee       0.82      0.76      0.79       100
       beetle       0.71      0.77      0.74       100
      bicycle       0.87      0.81      0.84       100
       bottle       0.82      0.78      0.80       100
         bowl       0.48      0.45      0.47       100
          boy       0.52      0.53      0.52       100
       bridge       0.73      0.84      0.78       100
          bus       0.72      0.73      0.73       100
    butterfly       0.73      0.61      0.66       100
        camel       0.67      0.81      0.73       100
          can       0.78      0.76      0.77       100
       ca

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [24]:
torch.save(model_source,'/content/gdrive/MyDrive/CS523/project_model_cifar100.ckpt')

Strategy: Batch Reordering<br>
Attack type: HightoLow 

In [ ]:
attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]
trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchReorderSampler(trainset, atk_type=attack_type[1]))

In [ ]:
from pytorch_lightning.loggers import WandbLogger
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
wandb.init(project="CIFAR-100_BatchReodering-Attack_1", entity="aashlessha")
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=200,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model_source)

In [ ]:
model_source.to(device)
model_source.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model_source(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))